In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install openpyxl

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import models

In [ ]:
path_data = "/content/drive/MyDrive/SuccKD_MultiSpecies/Data/"
path_model = "/content/drive/MyDrive/SuccKD_MultiSpecies/Model/"
path_result = "/content/drive/MyDrive/SuccKD_MultiSpecies/Result/"

In [ ]:
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [ ]:
# Tải dữ liệu test
file_test ="Test_E.coli_succ.csv"
df_test =pd.read_csv(path_data+file_test,delimiter= ',')
text_test =[] #PTMsequend kmer
for i in df_test['Peptide_WS_33']:
  temp = ProSentence(i,k)
  text_test.append(temp)
df_test['k_mer'] =text_test

test_sequences = []
for each in text_test:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    test_sequences.append(each_index_list)
data_token = []
for i in df_test['k_mer']:
   data_token.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token[1])
Xtest = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytest = np.array(df_test['Label'])
# perform one-hot encoding on the labels
ytest = np.array(ytest)
lb = LabelBinarizer()
ytest= lb.fit_transform(ytest)
ytest = to_categorical(ytest)
ytest.shape

(1670, 2)

In [ ]:
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE =300

In [ ]:
TIME_STEPS

33

In [ ]:
pip install openpyxl

In [ ]:
import numpy as np
import json
import os  # Import os to check if the path exists
import pandas as pd  # Import pandas for easy CSV saving
from sklearn.metrics import confusion_matrix  # Import confusion_matrix
from keras import models


model = models.load_model(path_model + "KD2_Succi.h5")
# Generate final predictions using logits from both students
ypred = model.predict(Xtest)

# Ensure predictions are in the correct format
ypred = np.argmax(ypred, axis=1)  # Getting the predicted class
ytest_true = np.argmax(ytest, axis=1)  # Getting the true class

# Compute the confusion matrix
confusion_mat = confusion_matrix(ytest_true, ypred)  # Rename to avoid overwriting

# Calculate TP, TN, FP, FN
TN = confusion_mat[0, 0]  # True Negative
FP = confusion_mat[0, 1]  # False Positive
FN = confusion_mat[1, 0]  # False Negative
TP = confusion_mat[1, 1]  # True Positive

# Create a DataFrame to hold TP, TN, FP, FN values
data = {'TP': [TP],
        'FP': [FP],
        'TN': [TN],
        'FN': [FN]}

df = pd.DataFrame(data)

# Define the path where you want to save the output file
# Make sure this points to a valid directory
output_file = "E.coli_KD2_Succ.xlsx"

# Write the DataFrame to an Excel file
df.to_excel(os.path.join(path_result, output_file), index=False)

print(f"Các giá trị TP, TN, FP, FN đã được ghi vào file {output_file} dưới dạng cột.")


53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Các giá trị TP, TN, FP, FN đã được ghi vào file E.coli_KD2_Succ.xlsx dưới dạng cột.
